<a href="https://colab.research.google.com/github/nikeshchaudhari/Chatbot/blob/main/Intelligent_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:


df = pd.read_csv('Ecommerce_FAQs.csv')

# Check and rename columns to 'question' and 'response' if needed
if 'prompt' in df.columns and 'response' in df.columns:
    df.rename(columns={'prompt': 'question'}, inplace=True)

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

class BERTChatBot:
    def __init__(self, data):
        # Initialize with the dataset (questions and responses)
        if 'question' not in data.columns or 'response' not in data.columns:
            raise KeyError("CSV file must contain 'question' and 'response' columns")

        # Store the questions and responses as a dictionary
        self.responses = {row['question']: row['response'] for _, row in data.iterrows()}
        self.chat_history = []

    def encode_text(self, text):
        """Encode the input text using BERT tokenizer and model."""
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze()

    def get_best_response(self, user_input):
        """Find the most similar response from predefined responses using cosine similarity."""
        user_input_embedding = self.encode_text(user_input).unsqueeze(0)

        best_score = -1
        best_response = None
        for question, reply in self.responses.items():
            response_embedding = self.encode_text(question).unsqueeze(0)
            similarity = cosine_similarity(user_input_embedding.numpy(), response_embedding.numpy())
            if similarity > best_score:
                best_score = similarity
                best_response = reply
        return best_response

    def chat(self):
        """Run the chat interface."""
        print("Intelligent Chatbot")
        while True:
            user_input = input(">> User: ")
            if user_input.lower() in ['bye', 'quit', 'exit']:
                print("BERT ChatBot: Goodbye!")
                break
            response = self.get_best_response(user_input)
            print(f">> BERT ChatBot: {response}")

# Start the chatbot with dataset loaded from CSV
chatbot = BERTChatBot(df)
chatbot.chat()


Intelligent Chatbot
>> User: hii
>> BERT ChatBot: Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.
